In [1]:
import os

In [2]:
from chembl_webresource_client.new_client import new_client
available_resources = [resource for resource in dir(new_client) if not resource.startswith('_')]
print(available_resources)

['activity', 'activity_supplementary_data_by_activity', 'assay', 'assay_class', 'atc_class', 'binding_site', 'biotherapeutic', 'cell_line', 'chembl_id_lookup', 'compound_record', 'compound_structural_alert', 'description', 'document', 'document_similarity', 'drug', 'drug_indication', 'drug_warning', 'go_slim', 'image', 'mechanism', 'metabolism', 'molecule', 'molecule_form', 'official', 'organism', 'protein_classification', 'similarity', 'source', 'substructure', 'target', 'target_component', 'target_relation', 'tissue', 'xref_source']


In [3]:
target_client = new_client.target
activity_client = new_client.activity

In [4]:
len(target_client), target_client[0].keys()

(15139,
 dict_keys(['cross_references', 'organism', 'pref_name', 'species_group_flag', 'target_chembl_id', 'target_components', 'target_type', 'tax_id']))

In [5]:
target_client[0]

{'cross_references': [{'xref_id': 'O43451',
   'xref_name': None,
   'xref_src': 'canSAR-Target'},
  {'xref_id': 'Maltase-glucoamylase',
   'xref_name': None,
   'xref_src': 'Wikipedia'}],
 'organism': 'Homo sapiens',
 'pref_name': 'Maltase-glucoamylase',
 'species_group_flag': False,
 'target_chembl_id': 'CHEMBL2074',
 'target_components': [{'accession': 'O43451',
   'component_description': 'Maltase-glucoamylase',
   'component_id': 434,
   'component_type': 'PROTEIN',
   'relationship': 'SINGLE PROTEIN',
   'target_component_synonyms': [{'component_synonym': '3.2.1.20',
     'syn_type': 'EC_NUMBER'},
    {'component_synonym': 'Alpha-1,4-glucosidase', 'syn_type': 'UNIPROT'},
    {'component_synonym': 'Maltase-glucoamylase', 'syn_type': 'UNIPROT'},
    {'component_synonym': 'MGA', 'syn_type': 'GENE_SYMBOL_OTHER'},
    {'component_synonym': 'MGAM', 'syn_type': 'GENE_SYMBOL'},
    {'component_synonym': 'MGAML', 'syn_type': 'GENE_SYMBOL_OTHER'},
    {'component_synonym': 'Synonyms=MGA', 

In [8]:
# Targets file
file_root = "./preprocessing/targets/"
target_ceartain_path = "Targets_HumanVoltage.csv"
target_certain_file_path = file_root + target_ceartain_path 

In [9]:
import pandas as pd
target_certain = pd.read_csv(target_certain_file_path, sep=',')

In [10]:
target_certain["ChEMBL ID"].iloc[0]

'CHEMBL1075319'

In [11]:
activity_client[0].keys()

dict_keys(['activity_comment', 'activity_id', 'activity_properties', 'assay_chembl_id', 'assay_description', 'assay_type', 'assay_variant_accession', 'assay_variant_mutation', 'bao_endpoint', 'bao_format', 'bao_label', 'canonical_smiles', 'data_validity_comment', 'data_validity_description', 'document_chembl_id', 'document_journal', 'document_year', 'ligand_efficiency', 'molecule_chembl_id', 'molecule_pref_name', 'parent_molecule_chembl_id', 'pchembl_value', 'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id', 'standard_flag', 'standard_relation', 'standard_text_value', 'standard_type', 'standard_units', 'standard_upper_value', 'standard_value', 'target_chembl_id', 'target_organism', 'target_pref_name', 'target_tax_id', 'text_value', 'toid', 'type', 'units', 'uo_units', 'upper_value', 'value'])

In [17]:
save_dir = "Targets_HumanVoltage/"

In [18]:
save_path = file_root + save_dir 

In [21]:
def mksure_path(dirs_or_files):
    if not os.path.exists(dirs_or_files):
        os.makedirs(dirs_or_files)

In [24]:
mksure_path(save_path)

In [48]:
def attain_data(target, stype, sunits, assay_type):
    data = activity_client.filter(target_chembl_id=target).filter(
        standard_type=stype,   # e.g. IC50
        standard_relation="=",
        standard_units=sunits,  # e.g. nM
        assay_type=assay_type,  # e.g. B
        data_validity_comment=None,
    ) 
    return data

In [13]:
target_certain["ChEMBL ID"].iloc[0]

'CHEMBL1075319'

In [49]:
temp = attain_data(target_certain["ChEMBL ID"].iloc[0], "IC50", "nM", "B")

In [38]:
temp[0]["assay_type"]

'B'

In [35]:
if temp[0]["data_validity_comment"] is None:
    print(1)

1


In [44]:
len(temp)

463

In [50]:
for idx, tmp in enumerate(temp):
    if tmp["data_validity_comment"] is not None:
        print(idx, tmp)

In [51]:
len(attain_data(target_certain["ChEMBL ID"].iloc[0], "IC50", "nM", "B"))

462

In [34]:
cnt = 0
for i in range(len(target_certain)):
    ic50_data = attain_data(target_certain["ChEMBL ID"].iloc[i], "IC50", "nM")
    if len(ic50_data) < 100:
        continue
    cnt += 1
    ic50_dataframe = pd.DataFrame.from_dict(ic50_data)
    ic50_dataframe.to_csv(save_path + "ic50_target" + str(cnt) + ".csv", index=False)

    break

In [10]:
# # IC50
# ic50_target1 = activity_client.filter(target_chembl_id=target_certain["ChEMBL ID"].iloc[0]).filter(
#     standard_type="IC50", 
#     standard_relation="=",
#     standard_units="nM",
# )

In [11]:
# len(ic50_target1)

486

In [12]:
# ic50_target1_dataframe = pd.DataFrame.from_dict(ic50_target1)

In [14]:
# ic50_target1_dataframe.to_csv(save_path + "ic50_target" + str(i) + ".csv", index=False)

In [ ]:
# # traverse targets and bioactivities
# herg_activities = activity.filter(target_chembl_id=herg['target_chembl_id']).filter(standard_type="IC50")